In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from warnings import simplefilter
warnings.filterwarnings("ignore", category =UserWarning,)

import gc
import pickle
from collections import OrderedDict
from pyproj import Transformer
import numpy as np
import pandas as pd
import iris
import glob
import sys
import os
import cartopy.crs as ccrs
import itertools
from scipy import spatial
import numpy.ma as ma
import tilemapbase
from math import cos, radians
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from pyproj import Proj, transform
import time

sys.path.insert(1,'../')
from Identify_Events_Functions import *
from Prepare_Data_Functions import *

pd.set_option('display.float_format', '{:.3f}'.format)
warnings.filterwarnings("ignore", category=UserWarning)

yrs_range= '2060_2081'
em = "bb222"
timeperiod = 'Future'
print(em)

if timeperiod == 'Future':
    sample_yr=2066
elif timeperiod == 'Present':
    sample_yr=2006
    
# Get Tb0 values at each gauge
tbo_vals = pd.read_csv('/nfs/a319/gy17m2a/PhD/datadir/RainGauge/interarrival_thresholds_CDD_noMissing.txt')
sample_cube = iris.load(f'/nfs/a319/gy17m2a/PhD/datadir/UKCP18_every30mins/2.2km_bng/{yrs_range}/{em}/bng_{em}a.pr{sample_yr}01.nc')[0][1,:,:]
gauge_num=1

bb222


In [2]:
base_dir = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/UKCP18_30mins/{timeperiod}/{em}/{gauge_num}/WholeYear"

In [3]:
# Get data 
pickle_file_filepath = f"/nfs/a319/gy17m2a/PhD/datadir/Gauge_Timeslices/{yrs_range}/{em}/gauge{gauge_num}_farFuture.nc"
full_year_cube = iris.load(pickle_file_filepath)[0]

Tb0, idx_2d = find_gauge_Tb0_and_location_in_grid(tbo_vals, gauge_num, sample_cube)

### This covers the full 20 years

In [4]:
# Get data 
pickle_file_filepath = f"/nfs/a319/gy17m2a/PhD/datadir/Gauge_Timeslices/{yrs_range}/{em}/gauge{gauge_num}_farFuture.nc"
full_year_cube = iris.load(pickle_file_filepath)[0]
data =full_year_cube.data

##### Filter cube according to different options
# Convert to dataframe
df = pd.DataFrame(data, columns=['precipitation (mm/hr)'])
df['times'] = full_year_cube.coord('time').units.num2date(full_year_cube.coord('time').points)
df['precipitation (mm)'] = df['precipitation (mm/hr)'] / 2   

In [34]:
def find_rainfall_core(df, duration, n_largest = 20):
    # Determine the length of the window based on provided duration
    window_length = int(duration * 2)  # Assuming data is in 30-min intervals

    # Calculate the rolling sum of precipitation over the specified window length
    df['Rolling_Sum'] = df['precipitation (mm)'].rolling(window=window_length).sum()

    # Identify the indices of the top 20 maximum rainfall accumulations
    top_indices = df['Rolling_Sum'].nlargest(n_largest).index.tolist()
    
    # Extract the rainfall cores corresponding to the top indices
    rainfall_cores = [df.iloc[idx:idx + window_length] for idx in top_indices]

    return rainfall_cores

def expand_events(rainfall_cores, df, Tb0):
    rainfall_events_expanded = []

    for rainfall_core in rainfall_cores:
        rainfall_core_after_search1 = search1(df, rainfall_core)
        rainfall_core_after_search2 = search2(df, rainfall_core_after_search1)
        rainfall_core_after_search3 = search3(df, rainfall_core_after_search2, Tb0=Tb0)
        
        # Check if the event has significant rainfall
        if len(rainfall_core_after_search3[rainfall_core_after_search3['precipitation (mm/hr)'] > 0.1]) > 0:
            # Remove leading dry values
            start_index = 0
            while start_index < len(rainfall_core_after_search3) and rainfall_core_after_search3['precipitation (mm/hr)'].iloc[start_index] < 0.1:
                start_index += 1
            
            # Remove trailing dry values
            end_index = len(rainfall_core_after_search3) - 1
            while end_index >= 0 and rainfall_core_after_search3['precipitation (mm/hr)'].iloc[end_index] < 0.1:
                end_index -= 1

            # Slice the DataFrame to remove leading/trailing dry values
            if start_index <= end_index:  # Ensure valid slicing
                rainfall_core_after_search3 = rainfall_core_after_search3.iloc[start_index:end_index + 1]

            rainfall_events_expanded.append(rainfall_core_after_search3)

    return rainfall_events_expanded

    
def find_top_n_rainfall_events(df, duration, Tb0, n_largest):
    # Find potential rainfall cores
    rainfall_cores = find_rainfall_core(df, duration, n_largest*2)
    
    top_events = []
    
    # Convert Tb0 from hours to timesteps (30 min = 0.5 hour)
    Tb0_timesteps = Tb0 * 2  # Tb0 in timesteps

    for core in rainfall_cores:
        # Expand events using the defined search functions
        expanded_event = expand_events([core], df, Tb0)

        # Check for overlaps within the Tb0 threshold
        overlaps = False
        for existing_event in top_events:
            # Check if any indices in the expanded event are within Tb0 timesteps of existing event indices
            for idx in expanded_event[0].index:
                # Calculate the range of indices for the existing event
                existing_event_range = existing_event.index
                # Check if the index is within Tb0 timesteps of any index in the existing event
                if any(abs(idx - ex_idx) <= Tb0_timesteps for ex_idx in existing_event_range):
                    overlaps = True
                    break
            if overlaps:
                # print("overlap")
                break  # Exit if any overlap is found

        # Only append if there's no overlap
        if not overlaps and expanded_event:
            top_events.append(expanded_event[0])  # Only append if there's no overlap

        # Limit the results to the top 20
        if len(top_events) >= n_largest:
            break

    return top_events


In [38]:
top_40_events = find_top_n_rainfall_events(df, duration=duration, Tb0=Tb0, n_largest= 40)
top_40_events[1]

,precipitation (mm/hr),times,precipitation (mm),Rolling_Sum
131785,2.399,2068-08-16 12:45:00,1.199,1.216
131786,1.404,2068-08-16 13:15:00,0.702,1.901
131787,4.929,2068-08-16 13:45:00,2.464,3.166
131788,3.362,2068-08-16 14:15:00,1.681,4.145
131789,5.354,2068-08-16 14:45:00,2.677,4.358
131790,1.982,2068-08-16 15:15:00,0.991,3.668
131791,2.468,2068-08-16 15:45:00,1.234,2.225
131792,1.284,2068-08-16 16:15:00,0.642,1.876
131793,2.560,2068-08-16 16:45:00,1.280,1.922
131794,0.322,2068-08-16 17:15:00,0.161,1.441
